<a href="https://colab.research.google.com/github/Pravin98651/FFCS-Helper-AI-Powered-Timetable-Faculty-Selection-for-FFCS/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import os
from google.colab import drive
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Mount Google Drive
drive.mount('/content/drive')

# Directory to save and load models
model_path = '/content/drive/My Drive/FFCS_Helper_Models/'
os.makedirs(model_path, exist_ok=True)

st.title("📅 FFCS Helper - Faculty Prediction and Timetable Optimization")

# Upload the CSV file
uploaded_file = st.file_uploader("Upload Faculty Data CSV", type=["csv"])
if uploaded_file:
    df = pd.read_csv(uploaded_file)
    st.write("### Faculty Data Preview")
    st.dataframe(df.head())

    # Data Preprocessing
    st.write("🔄 Data Preprocessing...")
    encoder = OneHotEncoder(sparse_output=False)
    dept_encoded = encoder.fit_transform(df[['Department']])
    dept_columns = encoder.get_feature_names_out(['Department'])
    df_dept = pd.DataFrame(dept_encoded, columns=dept_columns)
    df = pd.concat([df.drop('Department', axis=1), df_dept], axis=1)

    X = df.drop(columns=['EmpId'])
    y = df[['Teaching', 'Lenient', 'DA and Quiz', 'Marking']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train or Load Models
    models = {}
    categories = ['Teaching', 'Lenient', 'DA and Quiz', 'Marking']
    if not os.listdir(model_path):
        st.write("🚀 Training Models...")
        for category in categories:
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train[category])
            models[category] = model
            with open(os.path.join(model_path, f"{category}_model.pkl"), 'wb') as file:
                pickle.dump(model, file)
        with open(os.path.join(model_path, 'scaler.pkl'), 'wb') as file:
            pickle.dump(scaler, file)
        st.success("✅ Models Trained and Saved Successfully")
    else:
        st.write("📂 Loading Models from Google Drive...")
        for category in categories:
            with open(os.path.join(model_path, f"{category}_model.pkl"), 'rb') as file:
                models[category] = pickle.load(file)
        with open(os.path.join(model_path, 'scaler.pkl'), 'rb') as file:
            scaler = pickle.load(file)
        st.success("✅ Models Loaded Successfully")

    # Predict Best Faculty
    st.header("🔍 Predict Best Faculty")
    faculty_list = st.text_input("Enter Faculty IDs (comma separated):")
    prefs = {
        'Teaching': st.slider("Teaching Preference", 0.0, 10.0, 7.0),
        'Lenient': st.slider("Lenient Preference", 0.0, 10.0, 7.0),
        'DA and Quiz': st.slider("DA and Quiz Preference", 0.0, 10.0, 7.0),
        'Marking': st.slider("Marking Preference", 0.0, 10.0, 7.0)
    }

    if st.button("Predict Best Faculty"):
        if faculty_list:
            faculty_list = [int(x.strip()) for x in faculty_list.split(',')]
            filtered_faculty = df[df['EmpId'].isin(faculty_list)].copy()

            if filtered_faculty.empty:
                st.warning("Prediction not available. No faculty found in the given list.")
            else:
                faculty_features = filtered_faculty.drop(columns=['EmpId'])
                faculty_features_scaled = scaler.transform(faculty_features)
                predicted_scores = {cat: models[cat].predict(faculty_features_scaled) for cat in categories}
                for cat in categories:
                    filtered_faculty[f"Predicted_{cat}"] = predicted_scores[cat]

                best_faculty_per_category = {
                    category: filtered_faculty.loc[filtered_faculty[f"Predicted_{category}"].idxmax(), ['EmpId', f"Predicted_{category}"]].to_dict()
                    for category in categories
                }

                weight_vector = np.array([prefs.get(feature, 1) for feature in categories])
                weight_vector = weight_vector / weight_vector.sum()
                faculty_characteristics = filtered_faculty[[f"Predicted_{cat}" for cat in categories]].values
                custom_scores = np.dot(faculty_characteristics, weight_vector)
                filtered_faculty["Custom_Score"] = custom_scores
                ranked_faculty = filtered_faculty.sort_values(by="Custom_Score", ascending=False)
                best_overall_faculty = ranked_faculty.iloc[0][['EmpId', 'Custom_Score']].to_dict()

                st.subheader("🏆 Best Faculty Per Category")
                for category, faculty in best_faculty_per_category.items():
                    st.write(f"**{category}**: Faculty {faculty['EmpId']} (Score: {faculty[f'Predicted_{category}']:.2f})")

                st.subheader("🌟 Overall Best Faculty")
                st.write(f"**Faculty {best_overall_faculty['EmpId']}** (Custom Score: {best_overall_faculty['Custom_Score']:.2f})")

                st.subheader("📋 Full Faculty Ranking")
                st.dataframe(ranked_faculty[['EmpId', 'Custom_Score']])

        else:
            st.error("Please enter valid faculty IDs.")

    # Timetable Optimization
    st.header("🗓️ Timetable Optimization")
    course_type = st.selectbox("Select Course Type", ["Theory + Lab", "Theory Only", "Lab Only"])
    if course_type == "Theory + Lab":
        theory_slots = st.text_input("Enter Theory Slots (comma separated):")
        lab_slots = st.text_input("Enter Lab Slots (comma separated):")
    elif course_type == "Theory Only":
        theory_slots = st.text_input("Enter Theory Slots (comma separated):")
        lab_slots = ""
    elif course_type == "Lab Only":
        theory_slots = ""
        lab_slots = st.text_input("Enter Lab Slots (comma separated):")


    if st.button("Generate Timetable"):
        if (course_type in ["Theory + Lab", "Theory Only"] and not theory_slots) or (course_type in ["Theory + Lab", "Lab Only"] and not lab_slots):
            st.error("Please enter the required slots.")
        else:
            st.success("✅ Timetable Generated Successfully!")
            st.write(f"Course Type: {course_type}")
            if theory_slots:
                st.write(f"Theory Slots: {theory_slots}")
            if lab_slots:
                st.write(f"Lab Slots: {lab_slots}")


2025-03-11 06:33:08.117 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 06:33:08.118 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 06:33:08.120 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 06:33:08.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 06:33:08.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 06:33:08.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 06:33:08.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
! pip install streamlit -q